In [4]:
import tweepy
import pandas as pd
import numpy as np
import re
from googletrans import Translator
from google_trans_new import google_translator  
import datetime
import time
pd.options.display.max_columns = 999

In [2]:
# log = pd.read_csv("log_in.csv")
# consumer_key = log['key'][0]
# consumer_secret = log['key'][1]
# access_token = log['key'][2]
# access_token_secret = log['key'][3]
# auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
# auth.set_access_token(access_token, access_token_secret)
# api = tweepy.API(auth)
# keyword = 'ukraine'
# limit=5000
# # start_date = datetime.datetime(2022, 2, 24, 12, 00, 00)
# end_date = datetime.datetime(2023, 1, 23, 12, 00, 00)
# end_date = end_date.strftime("%Y-%m-%d")

# tweets = tweepy.Cursor(api.search_tweets, q=keyword, count=100, tweet_mode='extended', lang = 'ru', 
#                        result_type = 'mixed', until = end_date).items(limit)
# # tweets = api.search_tweets("ukraine", tweet_mode="extended", lang='ru')

# columns = ['User', 'Original Tweet']
# data = []

# #get tweet Id, location(city name, lang, long)
# for tweet in tweets:
#     data.append([tweet.user.screen_name, tweet.full_text])

# df = pd.DataFrame(data, columns=columns)

# print(df)


In [37]:
translator = Translator()

temp = df['Original Tweet'][1]

temp1 = translator.translate(temp).text
temp1



'Contractors of the 136th motorized rifle brigade of Dagestan refused to comply with command orders during the war in Ukraine and arbitrarily departed to the permanent deployment of the unit in Buinaksk.\n\nhttps://t.co/q8b4vswljc'

In [5]:
log = pd.read_csv("log_in.csv")
bearer_token = log['key'][4]


In [6]:
client = tweepy.Client(bearer_token, wait_on_rate_limit=True)

In [ ]:
hoax_tweets = []
for response in tweepy.Paginator(client.search_all_tweets, 
                                 query = 'ukraine -is:retweet lang:ru',
                                 user_fields = ['username', 'public_metrics', 'description', 'location'],
                                 tweet_fields = ['created_at', 'geo', 'public_metrics', 'text'],
                                 expansions = 'author_id',
                                 start_time = '2022-02-20T00:00:00Z',
                                 end_time = '2022-03-10T00:00:00Z',
                              max_results=500):
    time.sleep(1)
    hoax_tweets.append(response)



In [18]:
selected_tweets

[Response(data=[<Tweet id=1512875087178969100 text='https://t.co/s4qJcblx2p\nЕще 132 мирных жителя были найдены расстрелянными в украинском городе Макаров, который на 40% разрушен российской оккупацией, сообщил мэр.'>, <Tweet id=1512016877790408708 text='Украина: Подтверждение факта недозволенного обращения с военнопленными будет означать военное преступление | Human Rights Watch https://t.co/LlBWGoQqYJ'>, <Tweet id=1511298894923194372 text='Доклад Human Rights Watch о военных преступлениях, в том числе о происходившем в Буче\n(если кто вдруг считает, что HRW прозападная организация и действует против России, то вот их заявление о пытках российских пленных).\n\nhttps://t.co/YJAjQKO2zH'>, <Tweet id=1511074672531718149 text='✔ NYT: Президент США сегодня заявил, что сообщения о убийствах мирных жителей на Украине -  «военное преступление».  «Путен  жесток», — сказал Байден о президенте Путене.  «А то, что происходит в Буче, ужасно».  https://t.co/MjQZjbN6Z5 https://t.co/d2YcLiueQ9'>, <Twe

In [9]:
result = []
user_dict = {}
# Loop through each response object
for response in selected_tweets:
    # Take all of the users, and put them into a dictionary of dictionaries with the info we want to keep
    for user in response.includes['users']:
        user_dict[user.id] = {'username': user.username, 
                              'followers': user.public_metrics['followers_count'],
                              'tweets': user.public_metrics['tweet_count'],
                              'description': user.description,
                              'location': user.location
                             }
    for tweet in response.data:
        # For each tweet, find the author's information
        author_info = user_dict[tweet.author_id]
        # Put all of the information we want to keep in a single dictionary for each tweet
        result.append({'author_id': tweet.author_id, 
                       'username': author_info['username'],
                       'author_followers': author_info['followers'],
                       'author_tweets': author_info['tweets'],
                       'author_description': author_info['description'],
                       'author_location': author_info['location'],
                       'text': tweet.text,
                       'created_at': tweet.created_at,
                       'retweets': tweet.public_metrics['retweet_count'],
                       'replies': tweet.public_metrics['reply_count'],
                       'likes': tweet.public_metrics['like_count'],
                       'quote_count': tweet.public_metrics['quote_count']
                      })

# Change this list of dictionaries into a dataframe
df = pd.DataFrame(result)

In [10]:
df

,author_id,username,author_followers,author_tweets,author_description,author_location,text,created_at,retweets,replies,likes,quote_count
0,2477893675,SobornaUkraina,4654,87311,Соборна #Україна! Я люблю свою страну!,Украина,Российские солдаты - армия мародеров\n\n#Russi...,2022-03-02 10:06:19+00:00,1,2,3,0
1,65647594,StationCDRKelly,5289821,4996,Former NASA Astronaut & US Navy Captain @Keppl...,"Denver, CO",Это вероятные военные преступления не только П...,2022-03-01 23:42:01+00:00,28,2,184,0
2,837785295152115713,phillipsoninst,2597,108733,The Phillipson Institute\nForum to Educate and...,New Jersey,Главный прокурор Международного уголовного суд...,2022-03-01 03:00:45+00:00,0,1,0,0
3,2845262629,Rigsector,3654,100008,,Belgium,Россияне ударили по детскому садику кластерным...,2022-02-28 20:39:49+00:00,60,0,61,1
4,1472180907851513856,bukkugeito,5,428,"I am against war.\nOf course, I am also oppose...",None,В числе первых 137 жертв путинского вторжения ...,2022-02-25 10:07:22+00:00,0,0,0,0


In [22]:
# !pip install googletrans==4.0.0rc1

In [11]:
from googletrans import Translator
translator = Translator()
translated = translator.translate(text='nama saya', dest='en')
print(translated.text)

my name


In [13]:
translator = Translator()

translated_text = []

for text in df['text']:
    
    temp_text = translator.translate(text)
    print(temp_text.text)
    translated_text.append(temp_text.text)

Russian soldiers - the army of looters

#Russian Soldiers - An Army of #marauders

#war #UKRANE #CRIME #CRIMINALS https://t.co/49v7hqidw9
These are probable war crimes not only of Putin.These are war crimes of those who follow orders.According to international law, the armed forces are required to reject illegal orders.Everyone has a moral choice.
https://t.co/gucmrodh1j
The chief prosecutor of the International Criminal Court will begin the investigation of possible war crimes and crimes against humanity in Ukraine.Karim Khan said that the investigation will be conducted before the Russian invasion, but he intends to expand it.
https://t.co/1foqwjskey
The Russians hit the kindergarten cluster bombs.The child is killed.You are not on the street - you are an accomplice.https://t.co/fnke4smgsj https://t.co/wnbmmhycq
Among the first 137 victims of the Putin invasion were a child from Ukraine who died after a bombardment of a “war crime”
https://t.co/tkta07jhlk


In [14]:
len(translated_text)

5

In [16]:
df.shape

(5, 12)